## comp_vmodes
Compute vertical modes in NEMO, using nemodez package. (z-level formulation)

Compute, store and load.

In [1]:
%load_ext memory_profiler
%matplotlib widget
import matplotlib as mpl
from matplotlib import pyplot as plt
#from glob import glob
from pathlib import Path
import os

import numpy as np

import scipy.interpolate as itp
import xarray as xr
from xorca.lib import load_xorca_dataset
import gsw
from xgcm import Grid

if True:
    from nemodez import Vmodes, load_vmodes

In [2]:
from dask.distributed import Client, LocalCluster, wait
# Start a local cluster with 4 single-threaded (HDF is triggering errors
# if they are multi-threaded) workers which get only 1500 MiB of memory each.
cluster = LocalCluster(n_workers=4, threads_per_worker=1, memory_limit=1500e6,
                       silence_logs=50)
client = Client(address=cluster)
client

Client Scheduler: tcp://127.0.0.1:53666 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 5.59 GiB


## define path and load datasets

In [3]:
data_path = Path("/Users/nlahaye/tmp/eNATL60")

data_fname = "eNATL60NOE-BLBT02_y2010m01d01.1h_*_0-botm.nc"
grid_fname = ["mesh_*_eNATL60NOE_3.6.nc", "mask_eNATL60NOE_3.6.nc"]

data_files = list(data_path.glob(data_fname))
grid_files = sum([list(data_path.glob(fna)) for fna in grid_fname],[]) # trick to flatten: sum([list_of_list],[])

In [4]:
### Loading data from numerical run
ds_tot = load_xorca_dataset(data_files=data_files, aux_files=grid_files,
                              decode_cf=True, model_config="nest"
                             )
print("dataset is {:.1f} GB".format(ds_tot.nbytes/1e9))

ds_tot = ds_tot.isel(t=0)
ds_tot

dataset is 2.6 GB


<xarray.Dataset>
Dimensions:   (x_c: 121, x_r: 121, y_c: 139, y_r: 139, z_c: 300, z_l: 300)
Coordinates: (12/33)
  * z_c       (z_c) int64 1 2 3 4 5 6 7 8 9 ... 293 294 295 296 297 298 299 300
  * z_l       (z_l) float64 0.5 1.5 2.5 3.5 4.5 ... 296.5 297.5 298.5 299.5
  * y_c       (y_c) int64 1 2 3 4 5 6 7 8 9 ... 132 133 134 135 136 137 138 139
  * y_r       (y_r) float64 1.5 2.5 3.5 4.5 5.5 ... 136.5 137.5 138.5 139.5
  * x_c       (x_c) int64 1 2 3 4 5 6 7 8 9 ... 114 115 116 117 118 119 120 121
  * x_r       (x_r) float64 1.5 2.5 3.5 4.5 5.5 ... 118.5 119.5 120.5 121.5
    ...        ...
    e2f       (y_r, x_r) float64 dask.array<chunksize=(139, 121), meta=np.ndarray>
    e3w       (z_l, y_c, x_c) float64 dask.array<chunksize=(2, 139, 121), meta=np.ndarray>
    tmask     (z_c, y_c, x_c) int8 dask.array<chunksize=(2, 139, 121), meta=np.ndarray>
    umask     (z_c, y_c, x_r) int8 dask.array<chunksize=(2, 139, 121), meta=np.ndarray>
    vmask     (z_c, y_r, x_c) int8 dask.array<chunksize=(2, 139, 121), meta=np.ndarray>
    fmask     (z_c, y_r, x_r) int8 dask.array<chunksize=(2, 139, 121), meta=np.ndarray>
Data variables:
    vomecrty  (z_c, y_r, x_c) float32 dask.array<chunksize=(2, 139, 121), meta=np.ndarray>
    vosaline  (z_c, y_c, x_c) float32 dask.array<chunksize=(2, 139, 121), meta=np.ndarray>
    votemper  (z_c, y_c, x_c) float32 dask.array<chunksize=(2, 139, 121), meta=np.ndarray>
    vovecrtz  (z_l, y_c, x_c) float32 dask.array<chunksize=(2, 139, 121), meta=np.ndarray>
    vozocrtx  (z_c, y_c, x_r) float32 dask.array<chunksize=(2, 139, 121), meta=np.ndarray>

In [5]:
### Loading mean stratification profile
mean_file = data_path/"eNATL60NOE_mean_strat.nc"
ds_moy = xr.open_dataset(mean_file)
if "e3w" not in ds_moy:   
    ds_moy = ds_moy.assign_coords(e3w=ds_tot.e3w)
ds_moy

<xarray.Dataset>
Dimensions:   (x_c: 121, y_c: 139, z_c: 300, z_l: 300)
Coordinates: (12/14)
  * z_c       (z_c) int64 1 2 3 4 5 6 7 8 9 ... 293 294 295 296 297 298 299 300
  * y_c       (y_c) int64 1 2 3 4 5 6 7 8 9 ... 132 133 134 135 136 137 138 139
  * x_c       (x_c) int64 1 2 3 4 5 6 7 8 9 ... 114 115 116 117 118 119 120 121
    depth_c   (z_c) float64 ...
    llat_cc   (y_c, x_c) float32 dask.array<chunksize=(139, 121), meta=np.ndarray>
    llon_cc   (y_c, x_c) float32 dask.array<chunksize=(139, 121), meta=np.ndarray>
    ...        ...
    e3t       (z_c, y_c, x_c) float64 ...
    tmask     (z_c, y_c, x_c) int8 ...
    depth_l   (z_l) float64 dask.array<chunksize=(2,), meta=np.ndarray>
  * z_l       (z_l) float64 0.5 1.5 2.5 3.5 4.5 ... 296.5 297.5 298.5 299.5
    t         datetime64[ns] 2010-01-01T00:30:00
    e3w       (z_l, y_c, x_c) float64 dask.array<chunksize=(2, 139, 121), meta=np.ndarray>
Data variables:
    vosaline  (z_c, y_c, x_c) float32 ...
    votemper  (z_c, y_c, x_c) float32 ...
    Nsqr      (z_l, y_c, x_c) float32 ...

In [6]:
### Define xgcm grids
# WARNINGS: for vertical level: weights only for T-lev on T, u and v grid, and for w-lev on T_points
metrics = {
    ('X',): ['e1t', 'e1u', 'e1v', 'e1f'], # X distances
    ('Y',): ['e2t', 'e2u', 'e2v', 'e2f'], # Y distances
    ('Z',): ['e3t', 'e3u', 'e3v', 'e3w'], # Z distances
    #('X', 'Y'): [] # Areas TODO
}
ix, jy = slice(0,1), slice(0, 1)
sds = ds_tot.isel(x_c=ix, x_r=ix, y_c=jy, y_r=jy).chunk({"x_c":10, "y_c":10})
sdm = ds_moy.isel(x_c=ix, y_c=jy).chunk({"x_c":10, "y_c":10, "z_c":-1, "z_l":-1})
grid = Grid(sds, metrics=metrics, periodic=False)
grid

<xgcm.Grid>
Y Axis (not periodic, boundary=None):
  * center   y_c --> right
  * right    y_r --> center
Z Axis (not periodic, boundary=None):
  * center   z_c --> left
  * left     z_l --> center
X Axis (not periodic, boundary=None):
  * center   x_c --> right
  * right    x_r --> center

## Compute vertical modes
compute, store and load

In [11]:
vmods = Vmodes(sdm, grid, nmodes=10, free_surf=False, persist=True)
dm = vmods.ds
dm

<xarray.Dataset>
Dimensions:  (mode: 11, x_c: 1, y_c: 1, z_c: 300, z_l: 300)
Coordinates: (12/15)
  * z_c      (z_c) int64 1 2 3 4 5 6 7 8 9 ... 293 294 295 296 297 298 299 300
  * z_l      (z_l) float64 0.5 1.5 2.5 3.5 4.5 ... 295.5 296.5 297.5 298.5 299.5
  * y_c      (y_c) int64 1
  * x_c      (x_c) int64 1
    llat_cc  (y_c, x_c) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    llon_cc  (y_c, x_c) float32 dask.array<chunksize=(1, 1), meta=np.ndarray>
    ...       ...
    t        datetime64[ns] 2010-01-01T00:30:00
    e3w      (z_l, y_c, x_c) float64 dask.array<chunksize=(300, 1, 1), meta=np.ndarray>
    depth_c  (z_c) float64 dask.array<chunksize=(300,), meta=np.ndarray>
    tmask    (z_c, y_c, x_c) int8 dask.array<chunksize=(300, 1, 1), meta=np.ndarray>
    e3t      (z_c, y_c, x_c) float64 dask.array<chunksize=(300, 1, 1), meta=np.ndarray>
  * mode     (mode) int64 0 1 2 3 4 5 6 7 8 9 10
Data variables:
    Nsqr     (z_l, y_c, x_c) float32 dask.array<chunksize=(300, 1, 1), meta=np.ndarray>
    c        (y_c, x_c, mode) float64 dask.array<chunksize=(1, 1, 11), meta=np.ndarray>
    phi      (y_c, x_c, z_c, mode) float64 dask.array<chunksize=(1, 1, 300, 11), meta=np.ndarray>
    phiw     (y_c, x_c, z_l, mode) float64 dask.array<chunksize=(1, 1, 300, 11), meta=np.ndarray>
    norm     (y_c, x_c, mode) float64 dask.array<chunksize=(1, 1, 11), meta=np.ndarray>

In [9]:
### store modes and reload it
vmods.store("essai.zarr", mode="w")
del vmods
vmods = load_vmodes("essai.zarr")
vmods

Store vertical modes in /Users/nlahaye/working_on/eNATL60/essai.zarr


Vmode object with dimensions ('mode', 'x_c', 'y_c', 'z_c', 'z_l')
  Number of modes = 10
  Corresponding Dataset in self.ds 
  Options / parameters:
g=9.81e+00, eig_sigma=1.00e-01, siz_sparse=3.00e+01
free_surf=False, corr_N=True

In [12]:
dm.c.isel(mode=0).values

array([[nan]])

In [13]:
plt.figure()
dm.c.plot(marker="*", yscale="log")
plt.grid(True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
### check orthonormality
dm = dm.isel(x_c=0, y_c=0, mode=slice(10), missing_dims="ignore")

pnpm = (dm.phi*dm.phi.rename({"mode":"modm"})*dm.e3t).sum("z_c") / dm.norm.mean()
wnwm = dm.phiw/dm.c * (dm.phiw/dm.c).rename({"mode":"modm"})
wnwm_b = ( (wnwm*dm.Nsqr*dm.e3w).sum("z_l") + vmods.g*wnwm.isel(z_l=0) ) / dm.norm.mean()
wnwm_a = ( (grid.interp(wnwm*dm.Nsqr, "Z", boundary="fill", fill_value=0) * dm.e3t).sum("z_c") 
        + vmods.g * wnwm.isel(z_l=0) )/dm.norm.mean()

norm = mpl.colors.LogNorm(vmin=1e-6, vmax=1e0)
fig, axs = plt.subplots(1, 3, figsize=(8,2))
ax = axs[0]
hpc = ax.imshow(pnpm.values, norm=norm)
plt.colorbar(hpc, ax=ax)
ax = axs[1]
hpc = ax.imshow(wnwm_a.values, norm=norm)
plt.colorbar(hpc, ax=ax)
ax = axs[2]
hpc = ax.imshow(wnwm_b.values, norm=norm)
plt.colorbar(hpc, ax=ax)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
imod = -1

plt.figure()
dm.phiw.isel(mode=imod).plot(y="depth_l", marker=".")

I = (grid.interp(dm.phiw.isel(mode=imod)**2*dm.Nsqr, "Z", boundary="fill", fill_value=0) * dm.e3t).sum("z_c")
S = vmods.g*dm.phiw.isel(z_l=0,mode=imod)**2
res = (dm.c**2*dm.norm).isel(mode=imod)
I.values, S.values, res.values

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(array(21.06729357), array(2.9656448e-34), array(20.88059243))

In [17]:
modamp = vmods.project_w(sds.vovecrtz)

plt.figure()
sds.vovecrtz.isel(x_c=0, y_c=0).plot(y="depth_l", marker="+")
vmods.reconstruct_w(modamp).assign_coords(depth_l=sds.depth_l).isel(x_c=0, y_c=0).plot(y="depth_l", linestyle="--")
plt.grid(True)